# Retrieving texts and counting words

In this example, we'll retrieve the texts of speeches from a remote server and do a basic word count.

## Scenario

Let's say we want to know how many words Achilles speaks to each of his interlocutors. We can search the DICES database for the relevant speeches using the API. Then, to count the number of words, we'll have to retrieve the text of the speeches themselves. Since the DICES *Speech* objects include CTS URNS, we can request the passages from a remote server. 

## Preliminaries

### Install the client library

If you don't have the DICES client library, you can install it with **pip**:
```
pip install git+https://github.com/cwf2/dices-client.git
```

### The DICES API

First step is to instantiate a connection to the DICES api. Two optional arguments allow you to specify endpoints for the DICES database and the CTS server hosting the texts:

- `dices_api`: URL for the DICES API. Default is `https://fierce-ravine-99183.herokuapp.com/api/`, our development instance on Heroku. This is free-tier and annoyingly slow.
- `cts_api`: URL for the CTS server. Default is `https://scaife-cts.perseus.org/api/cts/`, which provides CTS access to (**I think**) the same Perseus texts as the [Scaife Viewer](https://scaife.perseus.org/).

To create the API instance with these defaults:

In [ ]:
from dicesapi import DicesAPI
api = DicesAPI()

### Matplotlib for figures

Let's also import **pyplot**, for drawing a simple bar graph of the results. Note the Jupyter magic `%matplotlib inline` to display the figure right in the notebook. Some people like `%matplotlib notebook` better — it gives you some fancier display options.

In [ ]:
from matplotlib import pyplot
%matplotlib inline

## Running the experiment

Here's the code for calculating Achilles' speech lengths by addressee.

### First, download the speeches

Using the API, we can search speeches using a set of key-value pairs [listed in greater detail here](https://cwf2.github.io/dices/2021/06/20/alpha.html). For now, JSON results from the API are paged, so if your search has a lot of results, you may have to wait for several pages to download.

<div class="alert alert-warning" style="margin:1em 2em">
    <p><strong>NB:</strong> Because the server is on Heroku's free tier, it take a minute to wake up when you first run a search. Subsequent tries are usually faster.</p>
</div>

In [ ]:
speeches = api.getSpeeches(spkr_name='Achilles')

### Count the words for each speech

This involves retrieving each passage from the CTS server, and extracting the plaintext of its contents. In this toy example, we split into words using Python's built-in `split()`, which just looks for whitespace. For more context-aware tokenization, use CLTK.

<div class="alert alert-warning" style="margin:1em 2em">
    <p>Behind the scenes, the <code>getCTS()</code> method shown here is calling <strong>MyCapytain</strong>'s <a href="https://mycapytain.readthedocs.io/en/latest/MyCapytain.classes.html?highlight=gettextualnode#MyCapytain.resolvers.prototypes.Resolver.getTextualNode"><code>getTextualNode()</code></a>, which returns a <a href="https://mycapytain.readthedocs.io/en/latest/MyCapytain.api.html#MyCapytain.resources.texts.local.capitains.cts.CapitainsCtsPassage">CapitainsCtsPassage</a> instance.</p>
</div>

In [ ]:
# initialize our counter
count = {}

# iterate over all speeches
for speech in speeches:
    
    # retrieve the passage from the remote library
    cts_passage = speech.getCTS()
    
    # extract the text and split into words
    plaintext = cts_passage.text
    n_words = len(plaintext.split())
    
    # tally the word counts for each addressee
    for addressee in speech.addr:
        name = addressee.name
        if name == 'Achilles':
            name = 'himself'
        count[name] = count.get(name, 0) + n_words

### Examine the results

🤔 Let's see whether it worked!

In [ ]:
for name in sorted(count):
    print(name, count[name])

### Make a simple graph with pyplot

Seems good. Let's visualize it with a simple bar chart.

In [ ]:
# data for the graph
names = sorted(count)
y_pos = range(len(names))
bars = [count[name] for name in names]

# create a new figure
fig, ax = pyplot.subplots(figsize=(8, 8))

# draw the bars
ax.barh(y_pos, bars, align='center')

# annotate the graph
ax.set_yticks(y_pos)
ax.set_yticklabels(names)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Number of Words')
ax.set_ylabel('Addressee')
ax.set_title('Length of Achilles\' speeches')